In [1]:
import pandas as pd
import re

Importing files

In [59]:
fil1 = "../data/ecoli_transcriptomics.xlsx"
data1 = pd.read_excel(fil1)
data1 = data1[pd.notnull(data1["NCBI_GENE_ID"])]

fil2 = "../data/proteomics_BSY.xlsx"
data2 = pd.read_excel(fil2)

fil3 = "../data/proteomics_ser.xlsx"
data3 = pd.read_excel(fil3)

fil4 = "../data/transcriptomics_ser.xlsx"
data4 = pd.read_excel(fil4)

fil5 = "../data/EC_genome.txt"
data5 = pd.read_csv(fil5, sep='\t')

fil6 = "../data/st_1_mutations.txt"
st1_mutations = pd.read_csv(fil6, sep='\t')

fil7 = "../data/st_2_mutations.txt"
st2_mutations = pd.read_csv(fil7, sep='\t')

fil8 = "../data/st_3_mutations.txt"
st3_mutations = pd.read_csv(fil8, sep='\t')

fil9 = "../data/st_4_mutations.txt"
st4_mutations = pd.read_csv(fil9, sep='\t')

fil10 = "../data/st_5_mutations.txt"
st5_mutations = pd.read_csv(fil10, sep='\t')

fil11 = "../data/gene_id_translation.txt"
pegnum_to_gene = pd.read_csv(fil11, sep='\t')
pegnum_to_gene["GENE_NAME"] = pegnum_to_gene.apply(lambda x: x["NCBI_GENE_ID"] if pd.notnull(x["NCBI_GENE_ID"]) else x["RAST_GENE_ID"], axis=1)

In [60]:
data5 = data5.drop(data5.index[-1])

Defining function extracting PEG (FigNum) number and adding it to a seperate column

In [5]:
def peg_parser(ori_id):
    test = ori_id.split('"')
    test2 = test[1].split('.')

    if test2[2] == "p eg" :
        gen_id = test2[3]
    elif test2[2] == "r na":
        gen_id = "NaN"

    else:
        print(len(test))
        print(ori_id)

        
    return gen_id

In [6]:
data5['FigNum']=data5['Qualifiers'].map(peg_parser)

In [7]:
data5

,Name,Type,Region,Qualifiers,FigNum
0,mannose-specific adhesin FimH,CDS,complement(2297236..2298150),"/db_xref=""SEED:fig|562.15553.p eg.2181""; /tran...",2181
1,S-adenosylmethionine:tRNA ribosyltransferase-i...,CDS,1148163..1149233,"/db_xref=""SEED:fig|562.15553.p eg.1075""; /db_x...",1075
2,UDP-N-acetylmuramate:L-alanyl-gamma-D-glutamyl...,CDS,573277..574650,"/db_xref=""SEED:fig|562.15553.p eg.524""; /trans...",524
3,LSU ribosomal protein L34p,CDS,607..747,"/db_xref=""SEED:fig|562.15553.p eg.1""; /db_xref...",1
4,Ribonuclease P protein component (EC 3.1.26.5),CDS,797..1123,"/db_xref=""SEED:fig|562.15553.p eg.2""; /db_xref...",2
5,Inner membrane protein translocase component Y...,CDS,1398..2993,"/db_xref=""SEED:fig|562.15553.p eg.3""; /transla...",3
6,GTPase and tRNA-U34 5-formylation enzyme TrmE,CDS,3099..4463,"/db_xref=""SEED:fig|562.15553.p eg.4""; /db_xref...",4
7,Tryptophanase (EC 4.1.99.1),CDS,5001..6416,"/db_xref=""SEED:fig|562.15553.p eg.5""; /db_xref...",5
8,Tryptophan-specific transport protein,CDS,6585..7754,"/db_xref=""SEED:fig|562.15553.p eg.6""; /transla...",6
9,Putative transport protein,CDS,7886..9061,"/db_xref=""SEED:fig|562.15553.p eg.7""; /transla...",7


In [8]:
test = 'fiG|1005566.5.peG.1'
test.split('.')

['fiG|1005566', '5', 'peG', '1']

In [9]:
pegnum_to_gene

,RAST_GENE_ID,NCBI_GENE_ID,GENE_NAME
0,fiG|1005566.5.peG.1,rpmH,rpmH
1,fiG|1005566.5.peG.10,yieF,yieF
2,fiG|1005566.5.peG.100,yigB,yigB
3,fiG|1005566.5.peG.1000,ykfB,ykfB
4,fiG|1005566.5.peG.1001,ykfF,ykfF
5,fiG|1005566.5.peG.1002,yafX,yafX
6,fiG|1005566.5.peG.1003,ykfG,ykfG
7,fiG|1005566.5.peG.1004,ykfH,ykfH
8,fiG|1005566.5.peG.1005,yafW,yafW
9,fiG|1005566.5.peG.1006,ykfI,ykfI


Defining function extracting figfam number

In [10]:
def rast_to_fig(ori_id):
    test2 = ori_id.split('.')

    if test2[2] == "peG" :
        gen_id = test2[3]
    elif test2[2] == "rnA":
        gen_id = "NaN"

    else:
        print(len(test))
        print(ori_id)

        
    return gen_id

In [11]:
pegnum_to_gene['FigNum'] = pegnum_to_gene['RAST_GENE_ID'].map(rast_to_fig)
print(len(pegnum_to_gene))
pegnum_to_gene = pegnum_to_gene[pegnum_to_gene["FigNum"] != "NaN"]
print(len(pegnum_to_gene))

print(len(set(pegnum_to_gene["FigNum"])))
print(len(set(pegnum_to_gene["GENE_NAME"])))

4522
4413
4413
4311


In [12]:
pegnum_to_gene

,RAST_GENE_ID,NCBI_GENE_ID,GENE_NAME,FigNum
0,fiG|1005566.5.peG.1,rpmH,rpmH,1
1,fiG|1005566.5.peG.10,yieF,yieF,10
2,fiG|1005566.5.peG.100,yigB,yigB,100
3,fiG|1005566.5.peG.1000,ykfB,ykfB,1000
4,fiG|1005566.5.peG.1001,ykfF,ykfF,1001
5,fiG|1005566.5.peG.1002,yafX,yafX,1002
6,fiG|1005566.5.peG.1003,ykfG,ykfG,1003
7,fiG|1005566.5.peG.1004,ykfH,ykfH,1004
8,fiG|1005566.5.peG.1005,yafW,yafW,1005
9,fiG|1005566.5.peG.1006,ykfI,ykfI,1006


In [13]:
print(len(pegnum_to_gene))
print(len(pegnum_to_gene["FigNum"].unique()))
print(len(pegnum_to_gene["RAST_GENE_ID"].unique()))

4413
4413
4413


In [14]:
fignum_to_genename = dict(zip(pegnum_to_gene["FigNum"], pegnum_to_gene["GENE_NAME"]))

### Adding GeneID to data 5

In [15]:
data5["GENE_NAME"] = data5["FigNum"].map(fignum_to_genename.get)
data5 = data5[data5["FigNum"] != "NaN"]
data5_B = data5.copy()

### Making a dictionary to look up the GeneID of corresponding gene to a specific region with mutation

In [16]:
data5_fig_region_dict = {}
for row in range(len(data5)):
    region = data5_B['Region'][row]
    if "complement" in region:
        region = re.search('\(([^)]+)', region).group(1)
    else:
        pass
    val = list(range(int(region.split('..')[0]), int(region.split('..')[1])+1))
    data5_fig_region_dict[data5_B['GENE_NAME'][row]]=val

In [17]:
# Invert the dictionary: Position to list of genes
data5_reg_fig_dict = {}
for gene, li in data5_fig_region_dict.items():
    for pos in li:
        data5_reg_fig_dict.setdefault(pos, []).append(gene)

In [19]:
data5_reg_fig_dict[1054347]

['yagI']

Making a list with the FigNum of the genes affected by mutation

In [16]:
mut_dict = {}
mutations = [st1_mutations, st2_mutations, st3_mutations,
            st4_mutations, st5_mutations]
len(mutations)
mut_list = mutations[1]
len(mut_list)

1045

In [77]:
mut_dict = {}
mutations = {"St1": st1_mutations, "St2": st2_mutations, "St3": st3_mutations,
            "St4": st4_mutations, "St5": st5_mutations}


for strain, mut_list in mutations.items():
    temp_liste = set()
    for i in range(len(mut_list)):
        
        try:
            val = int(mut_list["Region"][i])
        except ValueError:
            val = mut_list["Region"][i].split('^')[0]
        if val in data5_reg_fig_dict:
            temp_liste.update(data5_reg_fig_dict[val])
            #temp_liste.append([data5_reg_fig_dict[val], mut_list["Region"][i]])
    mut_dict[strain] = temp_liste

In [78]:
mut_dict["st2"]

KeyError: 'st2'

In [79]:
data1

,NCBI_GENE_ID,Unnamed: 1,St1_trans,St2_trans,St3_trans,St4_trans,St5_trans,function,evidence_codes,figfam
0,aaeA,fig|1005566.5.peg.3936,3.269841,4.936508,3.523810,4.095238,2.396825,Membrane fusion component of tripartite multid...,"idu(2);Multidrug_Resistance,_Tripartite_System...",FIG00001080
1,aaeB,fig|1005566.5.peg.3937,2.668478,3.940217,2.304348,3.054348,1.967391,FUSARIC ACID RESISTANCE PROTEIN FUSB / FUSARIC...,ff,FIG00634841
2,aaeR,fig|1005566.5.peg.3934,2.029412,3.098039,1.529412,2.289216,2.000000,LysR family transcriptional regulator QseA,idu(1);LysR-family_proteins_in_Escherichia_col...,FIG00001509
3,aas,fig|1005566.5.peg.3525,1.808889,2.456296,1.708148,2.362963,0.986667,2-acylglycerophosphoethanolamine acyltransfera...,icw(2);CBSS-364106.7.peg.3211 icw(3);CBSS-3641...,FIG01953891
4,aat,fig|1005566.5.peg.1551,2.638462,3.023077,1.430769,2.115385,1.461538,Leucyl/phenylalanyl-tRNA--protein transferase ...,isu;Protein_degradation,FIG00000748
5,abgA,fig|1005566.5.peg.2006,5.369048,4.380952,1.345238,2.535714,1.833333,Catalyzes the cleavage of p-aminobenzoyl-gluta...,isu;p-Aminobenzoyl-Glutamate_Utilization,FIG00001384
6,abgB,fig|1005566.5.peg.2005,5.094737,5.347368,1.768421,3.368421,2.168421,Catalyzes the cleavage of p-aminobenzoyl-gluta...,icw(3);p-Aminobenzoyl-Glutamate_Utilization,FIG00139150
7,abgR,fig|1005566.5.peg.2007,1.451524,0.994460,0.407202,0.698061,0.803324,"Regulatory protein (induces abgABT, used to ca...",isu;LysR-family_proteins_in_Escherichia_coli i...,FIG00008656
8,abgT,fig|1005566.5.peg.2004,3.373016,3.500000,1.222222,2.182540,1.658730,Aminobenzoyl-glutamate transport protein,icw(2);p-Aminobenzoyl-Glutamate_Utilization,FIG00010524
9,abrB,fig|1005566.5.peg.1386,0.680180,0.828829,0.382883,0.608108,1.763514,Putative transport protein,ff,FIG00453115


In [80]:
id_test = 'sp|P75780|FIU_ECOLI'

Defining a function extracting gene ID and adding it to a new column

In [81]:
def prot_ID_parser(ori_id):
    test = ori_id.split('|')
    if len(test) >= 3 :
        gen_id = test[2].split('_')[0]
    else:
        gen_id=ori_id
        
    return gen_id[:3].lower()+gen_id[3:]

In [82]:
data2['Gene_ID']=data2['Protein IDs'].map(prot_ID_parser)

Merging transcriptomic data for strain 1-5, 7-9)

In [83]:
data2

,Protein IDs,St7_prot,St8_prot,St9_prot,Gene_ID
0,sp|P75780|FIU_ECOLI,NaN,NaN,NaN,fiu
1,sp|P14175|PROV_ECOLI,NaN,NaN,NaN,proV
2,sp|P77667|SUFA_ECOLI,NaN,NaN,NaN,sufA
3,sp|P06960|OTC2_ECOLI,0.000000,1.800000,1.000000,otc2
4,CON__P13645,0.000000,0.750000,0.333333,con__P13645
5,CON__P07477,0.000000,6.000000,2.000000,con__P07477
6,sp|P64545|YFGG_ECOLI,0.000000,9.000000,4.000000,yfgG
7,sp|P31828|PQQL_ECOLI,0.000000,6.000000,3.000000,pqqL
8,sp|P42632|TDCE_ECOLI,NaN,NaN,NaN,tdcE
9,sp|P76272|YEBT_ECOLI,0.000000,0.750000,0.750000,yebT


In [84]:
dataframe_full = data1.merge(data2,how='left' , left_on="NCBI_GENE_ID", right_on="Gene_ID")

In [85]:
dataframe_full["Final_name"] = dataframe_full.apply(lambda x: x["NCBI_GENE_ID"] if pd.notnull(x["NCBI_GENE_ID"]) else x["Gene_ID"], axis=1)

In [93]:
dataframe_full[pd.isnull(dataframe_full["NCBI_GENE_ID"])]

,NCBI_GENE_ID,Unnamed: 1,St1_trans,St2_trans,St3_trans,St4_trans,St5_trans,function,evidence_codes,figfam,...,St7_prot,St8_prot,St9_prot,Gene_ID,Final_name,St1_muts,St2_muts,St3_muts,St4_muts,St5_muts


In [86]:
len(data1)

4214

In [87]:
len(data2)

1616

Adding genomic information to data table

In [90]:
for k in sorted(mut_dict):
    v = mut_dict[k]
    dataframe_full[k + "_muts"] = dataframe_full["NCBI_GENE_ID"].map(v.__contains__).astype("int")

In [94]:
len(dataframe_full) == len(set(dataframe_full["NCBI_GENE_ID"]))

False

In [95]:
for strain in ("St7", "St8", "St9"):
    dataframe_full[strain + "_muts"] = 0

In [92]:
dataframe_full.to_csv("../data/Dataframe_full.tsv", sep="\t")

In [96]:
bad_cols = ["Unnamed: 1", "evidence_codes", "figfam", "Gene_ID", "Final_name", "Protein IDs"]
final_df = dataframe_full.copy()
for col in bad_cols:
    del final_df[col]

In [97]:
final_df.to_csv("../data/final_data.tsv", sep="\t")

out_json = []

,NCBI_GENE_ID,St1_trans,St2_trans,St3_trans,St4_trans,St5_trans,function,Protein IDs,St7_prot,St8_prot,St9_prot,St1_muts,St2_muts,St3_muts,St4_muts,St5_muts,St7_muts,St8_muts,St9_muts
0,aaeA,3.269841,4.936508,3.523810,4.095238,2.396825,Membrane fusion component of tripartite multid...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1,aaeB,2.668478,3.940217,2.304348,3.054348,1.967391,FUSARIC ACID RESISTANCE PROTEIN FUSB / FUSARIC...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2,aaeR,2.029412,3.098039,1.529412,2.289216,2.000000,LysR family transcriptional regulator QseA,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,aas,1.808889,2.456296,1.708148,2.362963,0.986667,2-acylglycerophosphoethanolamine acyltransfera...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
4,aat,2.638462,3.023077,1.430769,2.115385,1.461538,Leucyl/phenylalanyl-tRNA--protein transferase ...,sp|P00509|AAT_ECOLI,1.126984,3.238095,1.174603,1,1,1,1,1,0,0,0
5,abgA,5.369048,4.380952,1.345238,2.535714,1.833333,Catalyzes the cleavage of p-aminobenzoyl-gluta...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
6,abgB,5.094737,5.347368,1.768421,3.368421,2.168421,Catalyzes the cleavage of p-aminobenzoyl-gluta...,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
7,abgR,1.451524,0.994460,0.407202,0.698061,0.803324,"Regulatory protein (induces abgABT, used to ca...",NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
8,abgT,3.373016,3.500000,1.222222,2.182540,1.658730,Aminobenzoyl-glutamate transport protein,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
9,abrB,0.680180,0.828829,0.382883,0.608108,1.763514,Putative transport protein,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [42]:
len(set(dataframe_full["NCBI_GENE_ID"]))

4067

In [43]:
len((dataframe_full["NCBI_GENE_ID"]))

4527

In [45]:
dataframe_full["NCBI_GENE_ID"].sort_values()

0       aaeA
1       aaeB
2       aaeR
3        aas
4        aat
5       abgA
6       abgB
7       abgR
8       abgT
9       abrB
10      accA
11      accB
12      accC
13      accD
14      aceA
15      aceB
16      aceE
17      aceF
18      aceK
19      ackA
20      acnA
21      acnB
22      acpP
23      acpS
24      acpT
25      acrA
26      acrB
27      acrD
28      acrE
29      acrF
        ... 
1817     NaN
1818     NaN
1819     NaN
1820     NaN
1821     NaN
1822     NaN
1823     NaN
1824     NaN
1825     NaN
1826     NaN
1827     NaN
1828     NaN
1829     NaN
1830     NaN
1831     NaN
1832     NaN
1833     NaN
1834     NaN
1835     NaN
1836     NaN
1837     NaN
1838     NaN
1839     NaN
1840     NaN
1841     NaN
1842     NaN
1843     NaN
1844     NaN
1845     NaN
1846     NaN
Name: NCBI_GENE_ID, dtype: object